<a href="https://colab.research.google.com/github/makoo700/Connect-Dating-App/blob/master/inventory_pythonAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Data Importation**

In [2]:
from google.colab import files
import pandas as pd
import numpy as np


In [3]:
### Import file from local storage
uploaded = files.upload()

Saving inventory.xlsx to inventory.xlsx


In [4]:
### Read the file as dataframe
df = pd.read_excel('inventory.xlsx')
df.head()

,Product_ID,Sale_Date,Category,Region,Quantity,Unit_Cost,Selling_Price,Discount
0,P0415,2023-05-28,Clothing,South,62,249.80,456.94,31.51
1,P0463,2023-06-26,Clothing,South,69,377.40,986.74,20.63
2,P0179,2023-05-31,Books,North,30,88.69,150.71,18.27
3,P0526,2023-05-03,Clothing,North,62,422.60,835.50,31.04
4,P0195,2023-06-24,Electronics,East,45,163.55,39.23,47.80


## Computing key business metrics

In [5]:
df['Revenue'] = df['Quantity'] * df['Selling_Price'] * (1 - df['Discount']/100)
df['Cost'] = df['Quantity'] * df['Unit_Cost']
df['Profit'] = df['Revenue'] - df['Cost']
df['ProfitMargin'] = (df['Profit'] / df['Revenue']) * 100
df.head()


,Product_ID,Sale_Date,Category,Region,Quantity,Unit_Cost,Selling_Price,Discount,Revenue,Cost,Profit,ProfitMargin
0,P0415,2023-05-28,Clothing,South,62,249.80,456.94,31.51,19403.408772,15487.60,3915.808772,20.181035
1,P0463,2023-06-26,Clothing,South,69,377.40,986.74,20.63,54039.112122,26040.60,27998.512122,51.811569
2,P0179,2023-05-31,Books,North,30,88.69,150.71,18.27,3695.258490,2660.70,1034.558490,27.996918
3,P0526,2023-05-03,Clothing,North,62,422.60,835.50,31.04,35721.969600,26201.20,9520.769600,26.652421
4,P0195,2023-06-24,Electronics,East,45,163.55,39.23,47.80,921.512700,7359.75,-6438.237300,-698.659639


### **Basic Descriptive Statistics**

In [8]:
print(df[['Revenue', 'Profit', 'ProfitMargin']].describe())

            Revenue        Profit  ProfitMargin
count    500.000000    500.000000    500.000000
mean   20122.175797   6413.896797    -58.186776
std    17703.441663  15821.250126    350.068610
min       52.238285 -35617.878736  -4319.854039
25%     5625.926946  -1896.644565    -36.446995
50%    14121.415332   3941.622794     27.753695
75%    31605.391638  12787.359582     63.024138
max    84764.226600  71371.326600     98.546398


In [9]:
# Grouping by category and computing metrics
category_summary = df.groupby('Category').agg(
    MeanProfitMargin = ('ProfitMargin','mean'),
    TotalRevenue = ('Revenue','sum'),
    NumTransactions = ('Product_ID','count')
)

category_summary


,MeanProfitMargin,TotalRevenue,NumTransactions
Category,,,
Books,-122.947075,1.837095e+06,102
Clothing,-45.108910,2.368518e+06,106
Electronics,-26.632584,1.910954e+06,97
Furniture,-35.179591,2.092209e+06,106
Toys,-61.335332,1.852312e+06,89


## **Highest Profit Margin**

In [10]:
df.groupby('Category')['ProfitMargin'].mean().sort_values(ascending=False)


,ProfitMargin
Category,
Electronics,-26.632584
Furniture,-35.179591
Clothing,-45.108910
Toys,-61.335332
Books,-122.947075


## **Frequent Discounts**

In [ ]:
high_discount_df = df[df['Discount'] > 40]
freq_region = high_discount_df['Region'].value_counts().idxmax()
avg_profit_high_disc = high_discount_df['Profit'].mean()

print(f"Region with most high discounts: {freq_region}")
print(f"Average profit for these transactions: {avg_profit_high_disc:.2f}")
# Explanation: This reveals if aggressive discounting is driving volume at the expense of profitability.

Region with most high discounts: East
Average profit for these transactions: 2013.37


## **Loss making transactions**

In [12]:
losses = df[df['Profit'] < 0]
loss_count = len(losses)


## *Most profitable month*

In [ ]:
# Convert OrderDate to datetime if not already
df['OrderDate'] = pd.to_datetime(df['Sale_Date'])

# Extract month
df['Month'] = df['OrderDate'].dt.month

# Compute monthly profit
monthly_profit = df.groupby('Month')['Profit'].sum().sort_values(ascending=False)

# Show the most profitable month
most_profitable_month = monthly_profit.idxmax()
most_profitable_value = monthly_profit.max()

print("Most profitable month in 2023:", most_profitable_month,
      "with profit:", most_profitable_value)


Most profitable month in 2023: 8 with profit: 524982.4757470001
